$$
\frac{d W}{d p} = p Q (p^2 + Q) + \left( \frac{2 p}{p^2 + Q} - \frac{1}{p} - \frac{i U^2 S \mu p^2}{4} \right) W - \frac{W^2}{p} 
$$

$$
W(\infty) = - \frac{4 i Q p}{U^2 S \mu} \quad \text{as} \ p \to \infty
$$

In [1]:
using Plots, DifferentialEquations, LaTeXStrings, Printf, SpecialFunctions, Base.MathConstants

# Define constants
μ = 1.0
k = 1.0
fun = 1.0

# Real-valued ODE function
function f_real(du, u, p, t)
    Q, U = p
    W_real, W_imag = u

    # Common terms
    denom = t^2 + Q
    term1_real = (2t) / denom - 1 / t
    term1_imag = -(U^2 * μ * t^2) / (4 * k^2 * fun^2)
    term2 = t * Q * denom

    # W^2 terms
    W_sq_real = W_real^2 - W_imag^2
    W_sq_imag = 2 * W_real * W_imag

    du[1] = term1_real * W_real - term1_imag * W_imag - W_sq_real / t + term2
    du[2] = term1_real * W_imag + term1_imag * W_real - W_sq_imag / t
end

# Q and U values
Q = 1e-2
U = 1.0 # U = 1.0 for M_A = S^(-1/2) # U = 0.04 for M_A = S^(-3/5)
Q_values = 10.0 .^ range(-4, -0.02, length=100)
U_values = 10.0 .^ range(-4, -0.02, length=100)

# Time span (integrate from large to small t)
tspan = (20.0, 1e-7)

function solve_and_compute_Δ(p1, p2)
    # Initial condition (real and imaginary parts)
    w0_real = 0.0
    w0_imag = (-4 * p2 * tspan[1]) / (p1^2 * μ)
    w0 = [w0_real, w0_imag]

    prob = ODEProblem(f_real, w0, tspan, (p1, p2))

    try
        # Try Vern7(), Tsit5(), Rodas5() -> this seems to give the best results
        sol = solve(prob, Rodas5(), reltol=1e-6, abstol=1e-6, maxiters=1e5)

        # Ensure we reached the end point
        if !(sol.retcode == ReturnCode.Success)
            @warn "Early termination for p1=$p1, p2=$p2 with retcode $(sol.retcode)"
            return complex(NaN, NaN)
        end

        t_min = minimum(sol.t)
        W_prime_real, W_prime_imag = sol(t_min, Val{1})
        W_prime = complex(W_prime_real, W_prime_imag)

        Δhat = π / W_prime
        return Δhat
    catch e
        @warn "Failed for parameters p1=$p1, p2=$p2: $e"
        try
            # Fallback solver
            sol = solve(prob, Tsit5(), reltol=1e-6, abstol=1e-6, maxiters=1e5)
            t_min = minimum(sol.t)
            W_prime_real, W_prime_imag = sol(t_min, Val{1})
            W_prime = complex(W_prime_real, W_prime_imag)
            return π / W_prime
        catch e2
            @warn "Fallback solver also failed for p1=$p1, p2=$p2: $e2"
            return complex(NaN, NaN)
        end
    end
end

# Compute results
Δhat_valuesU = [solve_and_compute_Δ(Q, U_val) for U_val in U_values]
Δhat_valuesQ = [solve_and_compute_Δ(Q_val, U) for Q_val in Q_values]

# Process results
function process_results(Δhat_values)
    real_part = [isnan(x) ? NaN : real(x) for x in Δhat_values]
    imag_part = [isnan(x) ? NaN : imag(x) for x in Δhat_values]
    log_real = log10.(abs.(real_part))
    log_imag = log10.(abs.(imag_part))
    return log_real, log_imag
end

logΔ_realU, logΔ_imagU = process_results(Δhat_valuesU);
logΔ_realQ, logΔ_imagQ = process_results(Δhat_valuesQ);

In [2]:
# Create the plot

theme(:dao)

p = plot(size=(800, 600), dpi=300)

# Plot real part (swapped x and y)
scatter!(p, log10.(Q_values), logΔ_realQ,
    label="Real part",
    markersize=6,
    marker=:circle,
    linestyle=:dash,
    linewidth=3)

# Plot imaginary part (swapped x and y)
scatter!(p, log10.(Q_values), logΔ_imagQ,
    label="Imaginary part",
    markersize=6,
    marker=:utriangle,
    linestyle=:dash,
    linewidth=3)

# First, compute the exact solution for single mode case
function exact_sm_coppi(Q)
    numerator = -π * gamma(Q^(3 / 2) / 4 - 1 / 4) * Q^(5 / 4)
    denominator = 8 * gamma(Q^(3 / 2) / 4 + 5 / 4)
    return numerator / denominator
end

# Compute exact solution for valid Q values
sm_coppi_exact = exact_sm_coppi.(Q_values)

# Plot the exact solution line (swapped x and y)
plot!(p, log10.(Q_values), log10.(abs.(sm_coppi_exact)),
    label="No Normal Flow",
    linewidth=5,
    linestyle=:solid)

# Add plot formatting with swapped labels
plot!(p,
    #title=L"Dispersion Relation$",
    ylabel=L"\log_{10}{(\Delta'/S^{1/3})}",  # Swapped y and x labels
    xlabel=L"\log_{10}(Q)",              # Swapped y and x labels
    legend=:best,
    titlefontsize=25,
    tickfontsize=20,
    legendfontsize=15,
    yguidefontsize=20,
    xguidefontsize=20,
    # grid=true,
    # gridstyle=:dash,
    # gridalpha=0.25,
    # minorgrid=false,
    left_margin=2 * Plots.mm,
    bottom_margin=2 * Plots.mm
)

# Display the plot
p

savefig(p, "disprel_cf_U1.png")

"/home/belster12/JuliaFiles/Ricatti/disprel_cf_U1.png"